In [3]:
import pandas as pd
import numpy as np

from IPython.display import Markdown, display
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import cluster, datasets
from sklearn.cluster import KMeans
import random
import pickle


In [4]:
#Importing required file


In [5]:
films = pd.read_csv('films_w_awards_cluster.csv', index_col =False)


/var/folders/0r/l856hq9x61zf5p8g8vxjs_dh0000gn/T/ipykernel_806/1553699301.py:1: DtypeWarning: Columns (25,26,27,28,29,30,31,32,33,34,35,36,37,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  films = pd.read_csv('films_w_awards_cluster.csv', index_col =False)


In [6]:
#Importing scaler and kmeans models


In [10]:
# Ask for number of movie names to enter
num_movies = int(input("How many people are you? (Enter a number between 1 and 5) "))

# validating number of inputs
while num_movies < 1 or num_movies > 5:
    print("Please enter a number between 1 and 5.")
    num_movies = int(input("How many people are you? (Enter a number between 1 and 5): "))

# Create an empty list to store movie names
movie_names = []

# Loop to get movie names based on the number entered
for i in range(num_movies):
    while True:
        movie_name = input("Please enter a movie that you love (original name) #{}: ".format(i+1))
        search_result = films[films['movie_name'].str.lower().str.contains(movie_name.lower(), case=False, na=False)]
        if len(search_result) == 0:
            print("Ups, it seems that we don't know that movie... Please try again.")
        else:
            movie_names.append(movie_name.lower())
            break

# Search for the movie names in the 'films' dataframe
movie_recommendations = []


for user_input in movie_names:
    search_result = films[films['movie_name'].str.lower().str.contains(user_input, case=False, na=False)]

    if len(search_result) > 0:
        # If multiple options found, ask user to choose one
        if len(search_result) > 1:
            print("We have found different movies for that name! Please choose one from the list:")
            for i, movie in enumerate(search_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']].iterrows(), start=1):
                print("{}. {}".format(i, movie[1]['movie_name']))

            # Continuously ask for input until a valid choice is provided
            while True:
                try:
                    chosen_movie_index = int(input("Enter the number corresponding to the movie you are referring to: "))
                    if chosen_movie_index > 0 and chosen_movie_index <= len(search_result):
                        chosen_movie_result = search_result.iloc[chosen_movie_index-1]
                        chosen_movie_name = chosen_movie_result['movie_name']
                        print("")
                        print("You have chosen: {}".format(chosen_movie_name))
                        print("")
                        break
                    else:
                        print("")
                        print("Invalid choice. Please select a number from the list.")
                        print("")
                        
                except ValueError:
                    print("")
                    print("Invalid choice. Please select a number from the list.")
                    print("")
        else:
            chosen_movie_result = search_result.iloc[0]
            chosen_movie_name = chosen_movie_result['movie_name']
            print("")
            print("You have chosen: {}".format(chosen_movie_name))
            print("")
        # Get the cluster of the chosen movie
        cluster = chosen_movie_result['cluster']
        # Get the year_film of the chosen movie
        year_film = chosen_movie_result['year_film']

        # Convert year_film to integer
        year_film = int(year_film)

        print("")
        print("Searched movie:")
        print(chosen_movie_result[['movie_name', 'year_film', 'director']])
        print("")
        
        # Filter films dataframe by cluster
        films_in_same_cluster = films[films['cluster'] == cluster]

        if len(films_in_same_cluster) > 0:
            # Filter films in same cluster with close year_film
            films_close_year = films_in_same_cluster[(films_in_same_cluster['year_film'] >= year_film - 5) & (films_in_same_cluster['year_film'] <= year_film + 5)]
            if len(films_close_year) > 0:
                # Remove chosen movie from the recommendations
                films_close_year = films_close_year[films_close_year['movie_name'] != chosen_movie_name]
                # Sort films by rating in descending order
                films_close_year = films_close_year.sort_values(by='rating', ascending=False)
                # Select the 20 highest rated films
                top_20_films = films_close_year.head(20)
                # Print a random film from the same cluster with close year_film
                random_film =  top_20_films.sample(n=1)
            else:
                # Print a random film from the same cluster
                random_film =  top_20_films.sample(n=1)
        else:
            # Print a random film from the entire dataframe
            random_film =  top_20_films.sample(n=1)

        # Extract required columns
        output = random_film[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes','description', 'movie_id']].copy()
        # Convert movie_id to string and concatenate with IMDb URL
        output['imdb_url'] = 'https://www.imdb.com/title/' + output['movie_id'].astype(str)

        # Append the recommendation to the list of movie recommendations
        movie_recommendations.append(output)

# Display the movie recommendations
if len(movie_recommendations) > 0:
    print("Movie Recommendations:")
    for i, recommendation in enumerate(movie_recommendations, start=1):
        print("{}. Movie Name: {}".format(i, recommendation['movie_name'].values[0]))
        print("   Year: {}".format(recommendation['year_film'].values[0]))
        print("   Director: {}".format(recommendation['director'].values[0]))
        print("   Runtime: {} mins".format(recommendation['runtime'].values[0]))
        print("   Rating: {}".format(recommendation['rating'].values[0]))
        print("   Votes: {}".format(recommendation['votes'].values[0]))
        print("   IMDb URL: {}".format(recommendation['imdb_url'].values[0]))
        print("   Description: {}".format(recommendation['description'].values[0]))
        print("")

else:
    print("No movie recommendations found.")


How many people are you? (Enter a number between 1 and 5) 2
Please enter a movie that you love (original name) #1: batman
Please enter a movie that you love (original name) #2: ojnnj
Movie not found. Please try again.
Please enter a movie that you love (original name) #2: titanic
We have found different movies for that name! Please choose one from the list:
1. Batman
2. Batman and Robin
3. James Batman
4. Scooby-Doo Meets Batman
5. Fight! Batman, Fight!
6. Alyas Batman en Robin
7. Batman Returns
8. Batman: Mask of the Phantasm
9. Batman Forever
10. Batman & Robin
11. Twisted Batman Theatre
12. Batman Begins
13. Batman: The Final Victory
14. Batman Untold
15. Batman Revealed
16. Batman: Crusader
17. Um Conto De Batman: Na Psicose Do Ventríloquo
18. Batman Beyond: Rising Knight
19. Batman City of Heros
20. Batman v Superman: Dawn of Justice (Ultimate Edition)
21. Batman v Superman: Dawn of Justice
22. Batman: The Killing Joke
23. Batman: Return of the Caped Crusaders
24. Batman: Anarchy


# top_20_films

In [11]:
top_20_films

,year_film,movie_name,runtime,rating,votes,osc_nominated_num,osc_won_num,cannes_won_num,director,actor1,...,osc_won_category11,osc_won_category12,cannes_won_category1,cannes_won_category2,cannes_won_category3,movie_id,director_id,star_id,description,cluster
21616,1957,The Bridge on the River Kwai,161,8.2,223011,1,7,0,David Lean,William Holden,...,NaN,NaN,NaN,NaN,NaN,tt0050212,/name/nm0000180/,"/name/nm0000034/,/name/nm0000027/,/name/nm0370...",British POWs are forced to build a railway bri...,24
19026,1954,On the Waterfront,108,8.1,156754,4,8,0,Elia Kazan,Marlon Brando,...,NaN,NaN,NaN,NaN,NaN,tt0047296,/name/nm0001415/,"/name/nm0000008/,/name/nm0001500/,/name/nm0002...",An ex-prize fighter turned New Jersey longshor...,24
18116,1953,From Here to Eternity,118,7.6,48175,5,8,0,Fred Zinnemann,Burt Lancaster,...,NaN,NaN,NaN,NaN,NaN,tt0045793,/name/nm0003593/,"/name/nm0000044/,/name/nm0001050/,/name/nm0000...","At a U.S. Army base in 1941 Hawaii, a private ...",24
22091,1958,Gigi,115,6.6,23421,0,9,0,Vincente Minnelli,Leslie Caron,...,NaN,NaN,NaN,NaN,NaN,tt0051658,/name/nm0591486/,"/name/nm0910199/,/name/nm0001989/,/name/nm0002...","Weary of the conventions of Parisian society, ...",24


In [31]:

# Ask for number of movie names to enter
while True:
    try:
        num_movies = int(input("How many people are you? (Enter a number between 1 and 5): "))
        # validating number of inputs
        if num_movies < 1 or num_movies > 5:
            print("Please enter a number between 1 and 5.")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter a number between 1 and 5.")


# Create an empty list to store movie names
movie_names = []

# Loop to get movie names based on the number entered
for i in range(num_movies):
    while True:
        movie_name = input("Please enter a movie that you love (original name) #{}: ".format(i+1))
        search_result = films[films['movie_name'].str.lower().str.contains(movie_name.lower(), case=False, na=False)]
        if len(search_result) == 0:
            print("Ups, it seems that we don't know that movie... Please try again.")
        else:
            movie_names.append(movie_name.lower())
            break

# Search for the movie names in the 'films' dataframe
movie_recommendations = []


for user_input in movie_names:
    search_result = films[films['movie_name'].str.lower().str.contains(user_input, case=False, na=False)]

    if len(search_result) > 0:
        # If multiple options found, ask user to choose one
        if len(search_result) > 1:
            print("We have found different movies for that name! Please choose one from the list:")
            for i, movie in enumerate(search_result[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes', 'movie_id']].iterrows(), start=1):
                print("{}. {}".format(i, movie[1]['movie_name']))

            # Continuously ask for input until a valid choice is provided
            while True:
                try:
                    chosen_movie_index = int(input("Enter the number corresponding to the movie you are referring to: "))
                    if chosen_movie_index > 0 and chosen_movie_index <= len(search_result):
                        chosen_movie_result = search_result.iloc[chosen_movie_index-1]
                        chosen_movie_name = chosen_movie_result['movie_name']
                        print("")
                        print("You have chosen: {}".format(chosen_movie_name))
                        print("")
                        break
                    else:
                        print("")
                        print("Invalid choice. Please select a number from the list.")
                        print("")
                        
                except ValueError:
                    print("")
                    print("Invalid choice. Please select a number from the list.")
                    print("")
        else:
            chosen_movie_result = search_result.iloc[0]
            chosen_movie_name = chosen_movie_result['movie_name']
            print("")
            print("You have chosen: {}".format(chosen_movie_name))
            print("")
        # Get the cluster of the chosen movie
        cluster = chosen_movie_result['cluster']
        # Get the year_film of the chosen movie
        year_film = chosen_movie_result['year_film']

        # Convert year_film to integer
        year_film = int(year_film)

        print("")
        print("Searched movie:")
        print(chosen_movie_result[['movie_name', 'year_film', 'director']])
        print("")
        
        # Filter films dataframe by cluster
        films_in_same_cluster = films[films['cluster'] == cluster]

        if len(films_in_same_cluster) > 0:
            # Filter films in same cluster with close year_film
            films_close_year = films_in_same_cluster[(films_in_same_cluster['year_film'] >= year_film - 5) & (films_in_same_cluster['year_film'] <= year_film + 5)]
            if len(films_close_year) > 0:
                # Remove chosen movie from the recommendations
                films_close_year = films_close_year[films_close_year['movie_name'] != chosen_movie_name]
                # Sort films by rating in descending order
                films_close_year = films_close_year.sort_values(by='rating', ascending=False)
                # Select the 20 highest rated films
                top_20_films = films_close_year.head(20)
                # Print a random film from the same cluster with close year_film
                random_film =  top_20_films.sample(n=1)
            else:
                # Print a random film from the same cluster
                random_film =  top_20_films.sample(n=1)
        else:
            # Print a random film from the entire dataframe
            random_film =  top_20_films.sample(n=1)

        # Extract required columns
        output = random_film[['movie_name', 'year_film', 'director', 'runtime', 'rating', 'votes','description', 'movie_id']].copy()
        # Convert movie_id to string and concatenate with IMDb URL
        output['imdb_url'] = 'https://www.imdb.com/title/' + output['movie_id'].astype(str)

        # Append the recommendation to the list of movie recommendations
        movie_recommendations.append(output)

# Display the movie recommendations
if len(movie_recommendations) > 0:
    print("Movie Recommendations:")
    for i, recommendation in enumerate(movie_recommendations, start=1):
        print("{}. Movie Name: {}".format(i, recommendation['movie_name'].values[0]))
        print("   Year: {}".format(recommendation['year_film'].values[0]))
        print("   Director: {}".format(recommendation['director'].values[0]))
        print("   Runtime: {} mins".format(recommendation['runtime'].values[0]))
        print("   Rating: {}".format(recommendation['rating'].values[0]))
        print("   Votes: {}".format(recommendation['votes'].values[0]))
        print("   IMDb URL: {}".format(recommendation['imdb_url'].values[0]))
        print("   Description: {}".format(recommendation['description'].values[0]))
        print("")

else:
    print("No movie recommendations found.")


How many people are you? (Enter a number between 1 and 5): 1
Please enter a movie that you love (original name) #1: nightmare christmas
Ups, it seems that we don't know that movie... Please try again.
Please enter a movie that you love (original name) #1: nightmare
We have found different movies for that name! Please choose one from the list:
1. Dublin Nightmare
2. Nightmare
3. Nightmare Castle
4. Nightmare in the Sun
5. Day of the Nightmare
6. Nightmare in Wax
7. The Devil's Nightmare
8. The House in Nightmare Park
9. It Happened at Nightmare Inn
10. Madeleine: Anatomy of a Nightmare
11. Nightmare Honeymoon
12. The Transformation: A Sandwich of Nightmares
13. Nightmare in Blood
14. Farewell Nightmare Theatre
15. Nightmare City
16. Butcher, Baker, Nightmare Maker
17. Runaway Nightmare
18. American Nightmare
19. Nightmares
20. A Nightmare on Elm Street 2: Freddy's Revenge
21. Summer Camp Nightmare
22. Nightmare Weekend
23. Nighty Night: Midnight Nightmares
24. Twisted Nightmare
25. Rock

KeyboardInterrupt: Interrupted by user

In [20]:
chosen_movie_name

'The Antman'

In [21]:
chosen_movie_result

year_film                                                                2002
movie_name                                                         The Antman
runtime                                                                    94
rating                                                                    4.6
votes                                                                     141
osc_nominated_num                                                           0
osc_won_num                                                                 0
cannes_won_num                                                              0
director                                                      Christoph Gampl
actor1                                                              Götz Otto
actor2                                                         Yasmina Filali
actor3                                                           Lars Rudolph
actor4                                                     Elisa

In [22]:
movie_recommendations

[                  movie_name  year_film        director  runtime  rating  \
 18116  From Here to Eternity       1953  Fred Zinnemann      118     7.6   
 
        votes                                        description   movie_id  \
 18116  48175  At a U.S. Army base in 1941 Hawaii, a private ...  tt0045793   
 
                                    imdb_url  
 18116  https://www.imdb.com/title/tt0045793  ,
       movie_name  year_film          director  runtime  rating  votes  \
 75530       Cure       1997  Kiyoshi Kurosawa      111     7.4  17279   
 
                                              description   movie_id  \
 75530  A frustrated detective deals with the case of ...  tt0123948   
 
                                    imdb_url  
 75530  https://www.imdb.com/title/tt0123948  ]

In [23]:
output

,movie_name,year_film,director,runtime,rating,votes,description,movie_id,imdb_url
75530,Cure,1997,Kiyoshi Kurosawa,111,7.4,17279,A frustrated detective deals with the case of ...,tt0123948,https://www.imdb.com/title/tt0123948
